In [1]:
# Getting input from a file
start_states = []
goal_states = []
heuristics = {}

with open("input_file.txt") as f:
        for (i,line) in enumerate(f):
            if(i==0):
                list1 = line.split(':')[1:]
                start_states = [list(map(int, state.split(','))) for state in list1]
            if(i==1):
                list1 = line.split(':')[1:]
                goal_states = [list(map(int, state.split(','))) for state in list1]
            if(i==2):
                list1 = (line.split(':'))[1].split(',')
                heuristics = {list1[0]:1,list1[1]:2}


In [2]:
print("Start states:",start_states,"\nGoal:",goal_states)
print("heuristic functions:",heuristics)

Start states: [[4, 5, 1, 2, 3, 8, 0, 6, 7], [0, 1, 3, 5, 2, 6, 4, 7, 8], [1, 2, 3, 5, 6, 0, 7, 8, 4]] 
Goal: [[1, 2, 3, 4, 5, 6, 7, 8, 0], [1, 2, 3, 5, 8, 6, 0, 7, 4]]
heuristic functions: {'Misplaced Tiles': 1, 'Manhattan Distance': 2}


In [3]:
class Puzzle:
    
    goal_state=None
    hn = None
    hn_value=None
    
    def __init__(self,state,parent,action,path_cost):     # initiater of class object
        
        self.state = state
        self.parent = parent
        self.action = action
        if parent:
            self.path_cost = parent.path_cost + path_cost
        else:
            self.path_cost = path_cost
        
        if self.hn == 1:
            self.manhattan_hn()
        elif self.hn == 2:
            self.misplaced_hn()

    def __str__(self):                        #representing class object as customized string
        
        return str(self.state[0:3])+'\n'+str(self.state[3:6])+'\n'+str(self.state[6:9])
    
    
    def manhattan_hn(self):                   #manhattan distance heuristic calculation
        
        self.hn_value=0
        for num in range(1,9):
            distance = abs(self.state.index(num) - self.goal_state.index(num))
            i = int(distance/3)
            j = int(distance%3)
            self.hn_value = self.hn_value +i +j
        
        
    def misplaced_hn(self):                    #no. of misplaced tiles heuristic calculation
        
        sum = 0
        for i in range(9):
            if(self.state[i] == 0): continue
            if(self.state[i] != self.goal_state[i]): sum+=1
        self.hn_value = sum

        
    def goal_test(self):                       #check goal_state reached or not
        
        if self.state == self.goal_state:
            return True
        return False

    
    @staticmethod
    def find_legal_actions(i,j):               #finds legal actions from a particular tile in a puzzle
        
        legal_action = ['U', 'D', 'L', 'R']
        if i == 0:                             # up is illegal
            legal_action.remove('U')
        elif i == 2:                           # down is illegal
            legal_action.remove('D')
        if j == 0:                             # left is illegal
            legal_action.remove('L')
        elif j == 2:                           # right is illegal
            legal_action.remove('R')
        return legal_action
    
    
    def generate_child(self):                   #generate childs of state
        
        children=[]
        x = self.state.index(0)                 #getting index of element 0 i.e blank tile in linear array
        i = int(x / 3)
        j = int(x % 3)
        legal_actions = self.find_legal_actions(i,j)

        for action in legal_actions:            #for every legal action creating a child with new state
            new_state = self.state.copy()
            if action == 'U':
                new_state[x], new_state[x-3] = new_state[x-3], new_state[x]
            elif action == 'D':
                new_state[x], new_state[x+3] = new_state[x+3], new_state[x]
            elif action == 'L':
                new_state[x], new_state[x-1] = new_state[x-1], new_state[x]
            elif action == 'R':
                new_state[x], new_state[x+1] = new_state[x+1], new_state[x]
            children.append(Puzzle(new_state,self,action,1))
        return children

    
    def find_solution(self):                    #find optimal path if goal_state reached
        
        solution = []
        solution.append(self.action)            #storing the action taken to come to this current state
        path = self
        while path.parent != None:              #backtracking to the start_state
            path = path.parent         
            solution.append(path.action)
        solution = solution[:-1]
        solution.reverse()
        return solution

In [4]:

def Hill_climb(initial_state,goal_state,hn):
    
    Puzzle.goal_state = goal_state               #setting the value of class variable goal_state
    Puzzle.hn = hn                               #setting the value of class variable hn(heuristic function preference)
    
    explored=[]                                  #list of states explored i.e. present in closed list
    start_node=Puzzle(initial_state,None,None,0) #initializing start_state object
    goal_node=Puzzle(goal_state,None,None,0)
    
    open_list = []                               #creating open list
    heuristics_computed = []
    heuristics_computed.append(start_node.hn_value)
    open_list.append((start_node.hn_value,start_node))#inserting tuple(hn_value,initial_state)

    t0 = time()
    count=0
    
    while len(open_list)>0:
        node=open_list[0][1]                       #accessing state element from list
        open_list.pop()                            #removing state element from list
        explored.append(node.state)                #inserting state element into closed/explored state
        
        if node.goal_test():                     #checking the state for goal_state
            print("\nPuzzle solved successfully")
            print("\nStart State")
            print(start_node)
            print("\nGoal State")
            print(node)
            opt_path = node.find_solution()
            print('\nOptimal path: ',opt_path)
            print('\nTotal no. of states explored : ',len(explored)-1)
            print('\nHeuristics estimated at each state in the path are: ',heuristics_computed)
            return
        
        elif time()-t0>20:
            print("\nPuzzle not solvable within time limit")
            print("\nStart State")
            print(start_node)
            print("\nGoal State")
            print(node)
            print('\nTotal no. of states explored before termination : ',len(explored)-1)
            print('\nHeuristics estimated at each state in the path are:',heuristics_computed)
            return
        
        children=node.generate_child()          #generating childs of state if its not a goal state
        count += len(children)
        prev_value = node.hn_value
        best_child = None
        l_c = 0
        
        for child in children:                  # for every child if not explored, insert it in the open list
            curr_value = child.hn_value
            if curr_value < prev_value:
                if curr_value != 0:
                    heuristics_computed.append(curr_value)
                min_value = curr_value
                best_child = child
                l_c+=1
                
        
        if(l_c == 0):
            print("\nPuzzle not solvable because local optima reached")
            print("\nStart State")
            print(start_node)
            print("\nGoal State")
            print(node)            
            print("\nLocal optima state\n")
            print(node)
            print('\nTotal no. of states explored before termination : ',len(explored)-1)
            print('\nHeuristics estimated at each state in the path are:',heuristics_computed)
            return
        open_list.append((best_child.hn_value,best_child))
    return

In [5]:
from time import time

start = start_states[1]
goal = goal_states[0]
h_1 = 'Misplaced Tiles'
h_2 = 'Manhattan Distance'

print("Heuristic function : ",h_1)

t0 = time()

Hill_climb(start, goal, heuristics[h_1])
t1 = time() - t0

print('\nTime taken: ', t1)

print()
print("Heuristic function : ",h_2)

Hill_climb(start, goal, heuristics[h_2])
t2 = time() - t0 -t1
print('\nTime taken: ', t2)


Heuristic function :  Misplaced Tiles

Puzzle solved successfully

Start State
[0, 1, 3]
[5, 2, 6]
[4, 7, 8]

Goal State
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Optimal path:  ['R', 'D', 'L', 'D', 'R', 'R']

Total no. of states explored :  6

Heuristics estimated at each state in the path are:  [6, 5, 4, 3, 2, 1]

Time taken:  0.0009899139404296875

Heuristic function :  Manhattan Distance

Puzzle solved successfully

Start State
[0, 1, 3]
[5, 2, 6]
[4, 7, 8]

Goal State
[1, 2, 3]
[4, 5, 6]
[7, 8, 0]

Optimal path:  ['R', 'D', 'L', 'D', 'R', 'R']

Total no. of states explored :  6

Heuristics estimated at each state in the path are:  [6, 5, 4, 3, 2, 1]

Time taken:  0.0009958744049072266
